<center>
    <hr> 
    <h1> 7º algoritimo: Classificação de Venda de Carros: </h1>
    <br>
    <hr> 
    <h2> Objetivo: A partir de 3 informações relacionadas ao carro (milhagem por ano, ano do modelo e preço), criar um algoritimo que classifique o carro como vendido ou não </h2>
    <br>
    <I> Feito por Yasmin Miranda </I>
    <hr> 
</center>

<center>
    <font size = "4" color = "#003666"> <B> Importando bibliotecas necessárias: </B> </font> 
</center>

In [1]:
#!pip install graphviz
#import graphviz
import pandas as pd
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

<center>
<font size = "4" color = "#003666"> <B> Criando a base de dados: </B> </font> 
</center>

In [2]:
uri = "https://gist.githubusercontent.com/guilhermesilveira/4d1d4a16ccbf6ea4e0a64a38a24ec884/raw/afd05cb0c796d18f3f5a6537053ded308ba94bf7/car-prices.csv"
dados = pd.read_csv(uri)
dados.head()
#coluna0: id do carro
#coluna2: milhagem por ano
#coluna3: ano do modelo
#coluna4: preço que o dono do carro quer vendê-lo
#coluna5: se foi vendido ou não

,Unnamed: 0,mileage_per_year,model_year,price,sold
0,0,21801,2000,30941.02,yes
1,1,7843,1998,40557.96,yes
2,2,7109,2006,89627.50,no
3,3,26823,2015,95276.14,no
4,4,7935,2014,117384.68,yes


In [3]:
#traduzir colunas para portugues
mapa = {
    "mileage_per_year":"milhagem_por_ano",
    "model_year":"ano_do_modelo",
    "price":"preco",
    "sold":"vendido"
}
dados = dados.rename(columns=mapa)

In [4]:
#trabalhar com respostas binarias:
troca = {
    "no":0,
    "yes":1
}
dados['vendido'] = dados.vendido.map(troca)
dados.head()

,Unnamed: 0,milhagem_por_ano,ano_do_modelo,preco,vendido
0,0,21801,2000,30941.02,1
1,1,7843,1998,40557.96,1
2,2,7109,2006,89627.50,0
3,3,26823,2015,95276.14,0
4,4,7935,2014,117384.68,1


Como o valor do carro está associado a sua idade, iremos adicionar essa informação na base de dados:

In [5]:
from datetime import datetime
ano_atual = datetime.today().year 
dados['idade_do_modelo'] = ano_atual - dados.ano_do_modelo
dados.head()

,Unnamed: 0,milhagem_por_ano,ano_do_modelo,preco,vendido,idade_do_modelo
0,0,21801,2000,30941.02,1,24
1,1,7843,1998,40557.96,1,26
2,2,7109,2006,89627.50,0,18
3,3,26823,2015,95276.14,0,9
4,4,7935,2014,117384.68,1,10


Como no Brasil utilizamos quiilometros, vamos converter a milhagem na base de dados:

In [6]:
dados['km_por_ano'] = dados.milhagem_por_ano * 1.6094
dados.head()

,Unnamed: 0,milhagem_por_ano,ano_do_modelo,preco,vendido,idade_do_modelo,km_por_ano
0,0,21801,2000,30941.02,1,24,35086.5294
1,1,7843,1998,40557.96,1,26,12622.5242
2,2,7109,2006,89627.50,0,18,11441.2246
3,3,26823,2015,95276.14,0,9,43168.9362
4,4,7935,2014,117384.68,1,10,12770.5890


In [7]:
dados = dados.drop(columns=["Unnamed: 0","milhagem_por_ano","ano_do_modelo"],axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,24,35086.5294
1,40557.96,1,26,12622.5242
2,89627.50,0,18,11441.2246
3,95276.14,0,9,43168.9362
4,117384.68,1,10,12770.5890


<center>
<font size = "4" color = "#003666"> <B> Treino Algoritmo Linear: </B> </font> 
</center>

In [8]:
x = dados[['preco','idade_do_modelo',"km_por_ano"]]
y = dados['vendido']

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

SEED = 5
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

modelo = LinearSVC()
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

Treinaremos com 7500 elementos e testaremos com 2500 elementos
A acurácia foi 46.88%


C:\Users\yasmi\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Iremos avaliar um algoritmo Dummy como referencia, cujas predições são 0 ou 1 respeitando apenas as proporções de 0 e 1 do treino, sem levar outras informações em consideração, para saber se nosso modelo é bom (acuracia maior que do dummy)

In [9]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier()

dummy_stratified.fit(treino_x,treino_y)

acuracia = dummy_stratified.score(teste_x,teste_y)*100

print("A acurácia do dummy stratified foi de %.2f%%" % acuracia)

A acurácia do dummy stratified foi de 58.00%


Necessário melhorar o algoritmo, visto que a acuracia de um modelo bobo é maior que a do LinearSVC.

<center>
<font size = "4" color = "#003666"> <B> Testando Algoritmo Não Linear: </B> </font> 
</center>

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

SEED = 5
np.random.seed(SEED)
raw_treino_x, raw_teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

scaler = StandardScaler()
scaler.fit(raw_treino_x)
treino_x = scaler.transform(raw_treino_x)
teste_x = scaler.transform(raw_teste_x)

modelo = SVC()
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

Treinaremos com 7500 elementos e testaremos com 2500 elementos
A acurácia foi 77.48%


Ganho de acurácia muito maior, comparado com o Linear e com o Dummy. Entretanto, não conseguimos dizer com muita clareza qual o peso de cada variavel na predição de venda.

<center>
<font size = "4" color = "#003666"> <B> Testando Algoritmos de Arvore de Decisao: </B> </font> 
</center>

In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

SEED = 5
np.random.seed(SEED)
raw_treino_x, raw_teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(raw_treino_x, treino_y)
previsoes = modelo.predict(raw_teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

Treinaremos com 7500 elementos e testaremos com 2500 elementos
A acurácia foi 76.84%


A acurácia foi maior que com o SVC, e é possivel identificar o que está fazendo o algoritmo tomar decisões, qual o peso de cada variavel e quais valores são mais provaveis de classificar o carro como vendido ou não. alem disso, é possivel definir a quantidade de etapas ate a decisao final 

In [12]:
#gerar grafico
#from sklearn.tree import export_graphviz
#import graphviz
#features = x.columns
#dot_data = export_graphviz(modelo, out_file=None,
#                          feature_names = features, 
#                           filled=True,rounded=True, class_names = ["não","sim"]) #export retorna o formato de um gráfico dotdata
#grafico = graphviz.Source(dot_data)
#grafico